In [10]:
import requests
import lxml.html as lh
import unicodecsv
from lxml.cssselect import CSSSelector
import requests_cache
import json
import itertools

requests_cache.install_cache('demo_cache')


In [13]:
s = requests.Session()
s.mount('http://', requests.adapters.HTTPAdapter(max_retries=10))

def read_lines(reader):
    headers = reader.next()
    for x in reader:
        yield dict(zip(headers,x))
    
def stringify_children(node, down=False):
    from lxml.etree import tostring
    from itertools import chain
    parts = ([node.text if hasattr(node, 'text') else node] +
            (list(chain(*([c.text if hasattr(c,'text') else c] + 
                          ([tostring(c,method="text",encoding="unicode")] if down and hasattr(c,'text') and not c.text else []) +
                          [c.tail if hasattr(c,'tail') else ''] for c in node.getchildren())))
             if hasattr(node, 'getchildren') else []) +
            [node.tail if hasattr(node, 'tail') else ''])
    # filter removes possible Nones in texts and tails
    return ''.join(filter(None, parts))
    
def extract_data(x,down=False):
    return stringify_children(x,down=down).strip()

def extract_line(tr):
    return [x
            for x in 
            tr.xpath('./*[name()="th" or name()="td"]')]


def extract_table(t):
    trs = t.xpath('.//tr')
    headers = [extract_data(x) for x in extract_line(trs[0])]
    ds = []
    for tr in trs[1:]:
         ds.append(dict(zip(headers,extract_line(tr))))
    return ds
    

In [14]:
root_url = 'https://data.fis-ski.com/global-links/all-fis-results.html?place_search=&seasoncode_search=all&sector_search=AL&date_search=&gender_search=&category_search=WSC&codex_search=&nation_search=&disciplinecode_search=&date_from=01&search=Search&limit=100'

root1 = lh.fromstring(s.get(root_url).content)
table = root1.xpath('//div[contains(@class,"bloc-tab")]/table[contains(@class,"footable")]')[0]

events = [x for x in extract_table(table) if len(x.keys())>1]
for event in events:
    keys = event.keys()
    print(keys)
    event['link'] = event['Event'].xpath('.//a/@href')[0]
    for k in keys:
        event[k] = extract_data(event[k],down=True)
    print(event['Place'])
    print(event['link'])

['Status', 'Discipline', 'Codex', 'Comments', 'NationNat.', 'Event', 'SectorSect.', 'GenderG.', 'Place', 'CategoryCat.', 'Date', 'Download']
St. Moritz
https://data.fis-ski.com/dynamic/event-details.html?event_id=39153&cal_suchsector=AL
['Status', 'Discipline', 'Codex', 'Comments', 'NationNat.', 'Event', 'SectorSect.', 'GenderG.', 'Place', 'CategoryCat.', 'Date', 'Download']
Vail / Beaver Creek
https://data.fis-ski.com/dynamic/event-details.html?event_id=34977&cal_suchsector=AL
['Status', 'Discipline', 'Codex', 'Comments', 'NationNat.', 'Event', 'SectorSect.', 'GenderG.', 'Place', 'CategoryCat.', 'Date', 'Download']
Schladming
https://data.fis-ski.com/dynamic/event-details.html?event_id=31721&cal_suchsector=AL
['Status', 'Discipline', 'Codex', 'Comments', 'NationNat.', 'Event', 'SectorSect.', 'GenderG.', 'Place', 'CategoryCat.', 'Date', 'Download']
Garmisch-Partenkirchen
https://data.fis-ski.com/dynamic/event-details.html?event_id=28178&cal_suchsector=AL
['Status', 'Discipline', 'Codex

In [15]:
root_url = 'https://data.fis-ski.com/global-links/all-fis-results.html?place_search=&seasoncode_search=all&sector_search=AL&date_search=&gender_search=&category_search=WSC&codex_search=&nation_search=&disciplinecode_search=&date_from=01&search=Search&limit=100'

root1 = lh.fromstring(s.get(root_url).content)
table = root1.xpath('//div[contains(@class,"bloc-tab")]/table[contains(@class,"footable")]')[0]

events = [x for x in extract_table(table) if len(x.keys())>1]
for event in events:
    keys = event.keys()
    event['link'] = event['Event'].xpath('.//a/@href')[0]
    for k in keys:
        event[k] = extract_data(event[k],down=True)
    print(event['Place'])
    print(event['link'])
    root2 = lh.fromstring(s.get(event['link']).content)
    table = root2.xpath('//div[contains(@class,"bloc-tab")]/table[contains(@class,"footable")]')[0]
    subevents = [x for x in extract_table(table) if 'Discipline' in x.keys()]
    for se in subevents:
        keys = se.keys()
        try:
            se['url'] = se['Discipline'].xpath('.//a/@href')[0]
            #print '    ', se['url']
        except IndexError:
            #print 'not found', se.keys(), lh.tostring(se['Discipline']), event['link']
            #print '   ', se.keys()
            pass
            #print 'could not find link'
        for k in keys:
            if k!='url':
                se[k] = extract_data(se[k],down=True)
        for k in event.keys():
            se['event_%s' % k] = event[k]
    event['subevents'] = subevents
    
all_subevents = list(itertools.chain(*[event['subevents'] for event in events]))
#print len(all_subevents)

St. Moritz
https://data.fis-ski.com/dynamic/event-details.html?event_id=39153&cal_suchsector=AL
Vail / Beaver Creek
https://data.fis-ski.com/dynamic/event-details.html?event_id=34977&cal_suchsector=AL
Schladming
https://data.fis-ski.com/dynamic/event-details.html?event_id=31721&cal_suchsector=AL
Garmisch-Partenkirchen
https://data.fis-ski.com/dynamic/event-details.html?event_id=28178&cal_suchsector=AL
Val d Isere
https://data.fis-ski.com/dynamic/event-details.html?event_id=24697&cal_suchsector=AL
Are
https://data.fis-ski.com/dynamic/event-details.html?event_id=22108&cal_suchsector=AL
Bormio/Santa Caterina
https://data.fis-ski.com/dynamic/event-details.html?event_id=15839&cal_suchsector=AL
St. Moritz
https://data.fis-ski.com/dynamic/event-details.html?event_id=9565&cal_suchsector=AL
St. Anton
https://data.fis-ski.com/dynamic/event-details.html?event_id=6424&cal_suchsector=AL
Vail/Beaver Creek, CO
https://data.fis-ski.com/dynamic/event-details.html?event_id=2540&cal_suchsector=AL
Sestrie

In [16]:
all_subevents[5]

{'': '',
 'Category': 'WSC',
 'Codex': '0210',
 'Comments': '',
 'Date': u'08.02.2017',
 'Discipline': 'Super G',
 'Download': u'Results (761.32 kb)Results, Analysis, Standings (7.82 MB)Start List (703.82 kb)',
 'Gender': u'M',
 'Nation': u'SUI',
 'Place': u'St. MoritzRUNCETLOCSTATUSRun 112:0012:00Official result',
 'event_CategoryCat.': 'TRA',
 'event_Codex': '',
 'event_Comments': 'Check changes',
 'event_Date': '6.-19.2.2017',
 'event_Discipline': '12xDH - 4xSL - 4xGS - SG - AC',
 'event_Download': '',
 'event_Event': '',
 'event_GenderG.': u'L                                    M',
 'event_NationNat.': u'SUI',
 'event_Place': 'St. Moritz',
 'event_SectorSect.': 'AL',
 'event_Status': u'',
 'event_link': 'https://data.fis-ski.com/dynamic/event-details.html?event_id=39153&cal_suchsector=AL',
 'url': 'https://data.fis-ski.com/dynamic/results.html?sector=AL&raceid=86878'}

In [17]:
[x['url'] for x in events[2]['subevents'] if 'url' in x]

['https://data.fis-ski.com/dynamic/results.html?sector=AL&raceid=71020',
 'https://data.fis-ski.com/dynamic/results.html?sector=AL&raceid=71022',
 'https://data.fis-ski.com/dynamic/results.html?sector=AL&raceid=71021',
 'https://data.fis-ski.com/dynamic/results.html?sector=AL&raceid=71024',
 'https://data.fis-ski.com/dynamic/results.html?sector=AL&raceid=71023',
 'https://data.fis-ski.com/dynamic/results.html?sector=AL&raceid=71026',
 'https://data.fis-ski.com/dynamic/results.html?sector=AL&raceid=71027',
 'https://data.fis-ski.com/dynamic/results.html?sector=AL&raceid=71025',
 'https://data.fis-ski.com/dynamic/results.html?sector=AL&raceid=71029',
 'https://data.fis-ski.com/dynamic/results.html?sector=AL&raceid=74601',
 'https://data.fis-ski.com/dynamic/results.html?sector=AL&raceid=71028',
 'https://data.fis-ski.com/dynamic/results.html?sector=AL&raceid=71031',
 'https://data.fis-ski.com/dynamic/results.html?sector=AL&raceid=71030',
 'https://data.fis-ski.com/dynamic/results.html?sec

In [18]:

done_urls = []
all_data = []


if True: #with open('/media/sf_ddj/geschichten/2016-10-26-zeit-SpA/skilaeufer/refined/subpages.csv') as f:
    l = all_subevents
    for i,x in enumerate(l):
        if not 'url' in x:
            continue
        if i%100==0:
            print(i)
        if x['url'] in done_urls:
            continue
        rq = s.get(x['url'])
        if rq.status_code!=200:
            print(x['url'])
            print(rq)
            continue
        print(x['url'])
        root = lh.fromstring(rq.content)
        tables = root.xpath('//div[contains(@class,"bloc-tab")]/table[contains(@class,"footable")]')
        if len(tables)!=1:
            print(x['url'], tables)
            continue
        
        tbl = tables[0]
        for d in extract_table(tbl):
            for k in d:
                #print k
                d[k] = extract_data(d[k],down=True)
                
             #d = dict(zip(headers,extract_data(tr)))
            d['source_url']=x['url']
            for k in x.keys():
                d['subevent_%s' % k] = x[k]
            all_data.append(d)
        done_urls.append(x['url'])
        
        if i%100==0:
            with open('done_urls_2.json','w') as f2:
                json.dump(done_urls, f2)

            with open('all_data_2.json','w') as f2:
                json.dump(all_data, f2)
                
with open('done_urls.json','w') as f2:
    json.dump(done_urls, f2)

with open('all_data.json','w') as f2:
    json.dump(all_data, f2)

0
https://data.fis-ski.com/dynamic/results.html?sector=AL&raceid=86873
https://data.fis-ski.com/dynamic/results.html?sector=AL&raceid=86874
https://data.fis-ski.com/dynamic/results.html?sector=AL&raceid=86876
https://data.fis-ski.com/dynamic/results.html?sector=AL&raceid=86875
https://data.fis-ski.com/dynamic/results.html?sector=AL&raceid=86877
https://data.fis-ski.com/dynamic/results.html?sector=AL&raceid=86878
https://data.fis-ski.com/dynamic/results.html?sector=AL&raceid=86879
https://data.fis-ski.com/dynamic/results.html?sector=AL&raceid=86882
https://data.fis-ski.com/dynamic/results.html?sector=AL&raceid=86880
https://data.fis-ski.com/dynamic/results.html?sector=AL&raceid=86881
https://data.fis-ski.com/dynamic/results.html?sector=AL&raceid=78928
https://data.fis-ski.com/dynamic/results.html?sector=AL&raceid=78929
https://data.fis-ski.com/dynamic/results.html?sector=AL&raceid=78930
https://data.fis-ski.com/dynamic/results.html?sector=AL&raceid=78933
https://data.fis-ski.com/dynamic

In [19]:
all_data[15]

{'Bib': u'16',
 'FIS Code': u'296472',
 'Name': u'FANCHINI Elena',
 'Nation': u'ITA',
 'Skis': 'Dynastar',
 'Year': u'1985',
 'source_url': 'https://data.fis-ski.com/dynamic/results.html?sector=AL&raceid=86873',
 'subevent_': '',
 'subevent_Category': 'TRA',
 'subevent_Codex': '5195',
 'subevent_Comments': 'Cancelled',
 'subevent_Date': u'06.02.2017',
 'subevent_Discipline': 'Downhill training',
 'subevent_Download': u'Extended start list (html)Start List Training 1 (352.61 kb)',
 'subevent_Gender': u'L',
 'subevent_Nation': u'SUI',
 'subevent_Place': u'St. Moritz',
 'subevent_event_CategoryCat.': 'TRA',
 'subevent_event_Codex': '',
 'subevent_event_Comments': 'Check changes',
 'subevent_event_Date': '6.-19.2.2017',
 'subevent_event_Discipline': '12xDH - 4xSL - 4xGS - SG - AC',
 'subevent_event_Download': '',
 'subevent_event_Event': '',
 'subevent_event_GenderG.': u'L                                    M',
 'subevent_event_NationNat.': u'SUI',
 'subevent_event_Place': 'St. Moritz',
 '

In [20]:
all_data[:5]

[{'Bib': u'1',
  'FIS Code': u'55970',
  'Name': u'SCHMIDHOFER Nicole',
  'Nation': u'AUT',
  'Skis': 'Fischer',
  'Year': u'1989',
  'source_url': 'https://data.fis-ski.com/dynamic/results.html?sector=AL&raceid=86873',
  'subevent_': '',
  'subevent_Category': 'TRA',
  'subevent_Codex': '5195',
  'subevent_Comments': 'Cancelled',
  'subevent_Date': u'06.02.2017',
  'subevent_Discipline': 'Downhill training',
  'subevent_Download': u'Extended start list (html)Start List Training 1 (352.61 kb)',
  'subevent_Gender': u'L',
  'subevent_Nation': u'SUI',
  'subevent_Place': u'St. Moritz',
  'subevent_event_CategoryCat.': 'TRA',
  'subevent_event_Codex': '',
  'subevent_event_Comments': 'Check changes',
  'subevent_event_Date': '6.-19.2.2017',
  'subevent_event_Discipline': '12xDH - 4xSL - 4xGS - SG - AC',
  'subevent_event_Download': '',
  'subevent_event_Event': '',
  'subevent_event_GenderG.': u'L                                    M',
  'subevent_event_NationNat.': u'SUI',
  'subevent_ev

In [21]:
#print len(all_data), len(done_urls)

In [22]:
all_keys = set()
for x in all_data:
    for k in x.keys():
        all_keys.add(k)

In [23]:
done_urls

['https://data.fis-ski.com/dynamic/results.html?sector=AL&raceid=86873',
 'https://data.fis-ski.com/dynamic/results.html?sector=AL&raceid=86874',
 'https://data.fis-ski.com/dynamic/results.html?sector=AL&raceid=86876',
 'https://data.fis-ski.com/dynamic/results.html?sector=AL&raceid=86875',
 'https://data.fis-ski.com/dynamic/results.html?sector=AL&raceid=86877',
 'https://data.fis-ski.com/dynamic/results.html?sector=AL&raceid=86878',
 'https://data.fis-ski.com/dynamic/results.html?sector=AL&raceid=86879',
 'https://data.fis-ski.com/dynamic/results.html?sector=AL&raceid=86882',
 'https://data.fis-ski.com/dynamic/results.html?sector=AL&raceid=86880',
 'https://data.fis-ski.com/dynamic/results.html?sector=AL&raceid=86881',
 'https://data.fis-ski.com/dynamic/results.html?sector=AL&raceid=78928',
 'https://data.fis-ski.com/dynamic/results.html?sector=AL&raceid=78929',
 'https://data.fis-ski.com/dynamic/results.html?sector=AL&raceid=78930',
 'https://data.fis-ski.com/dynamic/results.html?sec

In [26]:
all_keys = sorted(list(all_keys))

In [25]:
with open('fin_output_2.csv','w') as of:
    ow = unicodecsv.writer(of)
    ow.writerow(all_keys)
    for x in all_data:
        ow.writerow([x.get(k,'?????') for k in all_keys])